In [2]:
import os
os.chdir('../')

In [3]:
import mlflow
import joblib
from src.utils.utlis import *
from src.constants import *
import pandas as pd

In [4]:

os.environ['MLFLOW_TRACKING_URI']='https://dagshub.com/AIAkashMukherjee/House-Sale-Price-Prediction-.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME']='AIAkashMukherjee'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '588b7f4279c32acc1263e6c2cfbbd743a7f77705'

In [14]:
model=joblib.load('artifacts/model_trainer/model.pkl')

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    model_path:Path
    test_data:Path
    mlflow_uri: str
  

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def eval_config(self):
        eval_confg=EvaluationConfig(
            model_path="artifacts/model_trainer/model.pkl",
            test_data="artifacts/data_transformation/final_test.csv",
            mlflow_uri=os.environ['MLFLOW_TRACKING_URI'],
           
        )    
        return eval_confg

In [19]:
class ModelEvaluation:
    def __init__(self,config:EvaluationConfig):
        self.config=config
        self.scores={}

    def save_score(self):
        save_json(path=Path("scores.json"), data=self.scores) 

    def evaluate_model(self):
            # train_data=pd.read_csv(self.config.training_data_path)
            test_data=pd.read_csv(self.config.test_data)
            X_test = test_data.iloc[:, :-1].values
            y_test = test_data.iloc[:, -1].values   

            model=joblib.load(self.config.model_path)
            y_pred=model.predict(X_test)

            r2 = r2_score(y_test, y_pred)
            self.scores["R2 Score"] = r2
            return model

    def log_with_mlflow(self,model):
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        with mlflow.start_run():
            mlflow.log_param("Model Path", self.config.model_path)
            mlflow.log_metric("R2 Score", self.scores["R2 Score"])

            mlflow.sklearn.log_model(model, "random_forest_model")

    def evaluation(self):
        model = self.evaluate_model()

        # Save scores locally
        self.save_score()

        # Log evaluation with MLflow
        self.log_with_mlflow(model)
            

In [20]:
try:
    config = ConfigurationManager()
    eval_config = config.eval_config()
    evaluation = ModelEvaluation(eval_config)
    evaluation.evaluation()

except Exception as e:
   raise e

2024/11/17 17:52:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/17 17:52:06 INFO mlflow.tracking._tracking_service.client: 🏃 View run angry-trout-262 at: https://dagshub.com/AIAkashMukherjee/House-Sale-Price-Prediction-.mlflow/#/experiments/0/runs/6c8df6e9daa94e2d8caeef2f0e3cb7e3.
2024/11/17 17:52:06 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/AIAkashMukherjee/House-Sale-Price-Prediction-.mlflow/#/experiments/0.
